In [40]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [41]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("Resources/lending_data.csv")
lending_data_df = pd.read_csv(file_path)

# Review the DataFrame
lending_data_df.head(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
5,10100.0,7.438,50600,0.407115,4,1,20600,0
6,10300.0,7.490,51100,0.412916,4,1,21100,0
7,8800.0,6.857,45100,0.334812,3,0,15100,0
8,9300.0,7.096,47400,0.367089,3,0,17400,0
9,9700.0,7.248,48800,0.385246,4,0,18800,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [42]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_data_df["loan_status"]

# Separate the X variable, the features
X = lending_data_df.drop(columns=["loan_status"])

In [43]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [44]:
# Review the X variable DataFrame
X.head(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
5,10100.0,7.438,50600,0.407115,4,1,20600
6,10300.0,7.490,51100,0.412916,4,1,21100
7,8800.0,6.857,45100,0.334812,3,0,15100
8,9300.0,7.096,47400,0.367089,3,0,17400
9,9700.0,7.248,48800,0.385246,4,0,18800


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [45]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [106]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [107]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [108]:
# Make a prediction using the testing data
test_pred = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [109]:
# Print the balanced_accuracy score of the model

print(f"Accuracy score: {balanced_accuracy_score(y_test, test_pred)}")

Accuracy score: 0.9442676901753825


In [110]:
# Generate a confusion matrix for the model
test_matrix = confusion_matrix(y_test, test_pred)
print(test_matrix)

[[18679    80]
 [   67   558]]


In [111]:
# Print the classification report for the model
target_names = ["Low-Risk loan", "High-Risk loan"]
class_report = classification_report(y_test, test_pred,target_names=target_names, digits = 4)
print(class_report)

                precision    recall  f1-score   support

 Low-Risk loan     0.9964    0.9957    0.9961     18759
High-Risk loan     0.8746    0.8928    0.8836       625

      accuracy                         0.9924     19384
     macro avg     0.9355    0.9443    0.9398     19384
  weighted avg     0.9925    0.9924    0.9925     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** `According to the classification report, the low-risk (0) and high-risk (1) labels were predicted well by the logistic regression model with high precision, recall, and F1-scores achieved for both classes. The precision of 0.9964 and recall of 0.9957 were achieved for the low-risk loan class, while the high-risk loan class achieved a precision of 0.8746 and recall of 0.8928. The model obtained an accuracy of 0.9924 and an F1-score of 0.9925, indicating strong performance in predicting both healthy and high-risk loans.`

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [112]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversample = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = oversample.fit_resample(X_train, y_train)

In [113]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

1    56277
0    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [114]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
pred_resampled = classifier.predict(X_resampled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [115]:
# Print the balanced_accuracy score of the model 
score = balanced_accuracy_score(y_resampled, pred_resampled)
print(f"Balanced Accuracy Score: {score:.4f}")

Balanced Accuracy Score: 0.9942


In [116]:
# Generate a confusion matrix for the model
pred_matrix = confusion_matrix(y_resampled, pred_resampled)
print(pred_matrix)

[[55945   332]
 [  323 55954]]


In [117]:
# Print the classification report for the model
target_names = ["Low-Risk loan", "High-Risk loan"]
resampled_report = classification_report(y_resampled, pred_resampled,target_names=target_names, digits = 4)
print(resampled_report)

                precision    recall  f1-score   support

 Low-Risk loan     0.9943    0.9941    0.9942     56277
High-Risk loan     0.9941    0.9943    0.9942     56277

      accuracy                         0.9942    112554
     macro avg     0.9942    0.9942    0.9942    112554
  weighted avg     0.9942    0.9942    0.9942    112554



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** `The oversampling technique used in this logistic regression model seems to have improved its ability to predict high-risk loans, with near-perfect precision, recall, and F1-scores achieved for this class. However, while there was a slight decrease in precision, recall, and F1-score for the Low-Risk class compared to original model, the oversampling approach still achieved high scores for precision, recall, and F1-score for this class. The The resampled data appears to have helped the model identify more patterns in the data and make better predictions. The balanced accuracy score of 0.9942, confusion matrix, and classification report show that the model has strong performance in predicting both healthy and high-risk loans. Overall, it appears that the oversampling technique has been effective in improving the model's performance.`